In [1]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time

from IPython import display

In [2]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256)  # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model

In [4]:
model = make_generator_model()

In [ ]:
class Hidden(tf.keras.layers.Layer):
    def __init__(self, nLayers, d_model, d_output, dropout_rate=0.1):
        super().__init__()
        dims = [d_model] * nLayers
        dims = dims + [d_model * 2 + round( (d_output - d_model * 2) * (layer+1) / (nLayers) ) for layer in range(nLayers)]
        layers = [tf.keras.layers.Dense(dim, kernel_initializer=tf.keras.initializers.LecunUniform(), activation='relu') for dim in dims]
        self.seq = tf.keras.Sequential(layers)
    def call(self, x):
        x = self.seq(x)
        return x

In [ ]:
class Output(tf.keras.layers.Layer):
    def __init__(self, nLayers, d_model, d_output, dropout_rate=0.1):
        super().__init__()
        dims = [d_model] * nLayers
        dims = dims + [d_model * 2 + round( (d_output - d_model * 2) * (layer+1) / (nLayers) ) for layer in range(nLayers)]
        layers = [tf.keras.layers.Dense(dim, kernel_initializer=tf.keras.initializers.LecunUniform(), activation='relu') for dim in dims]
        self.seq = tf.keras.Sequential(layers)
    def call(self, x):
        x = self.seq(x)
        return x

In [ ]:
class Recurrent(tf.keras.Model):
    def __init__(self, dropout_rate=0.1):
        super().__init__()
        self.hiddenNN = Hidden(3, 8, 5, dropout_rate=0.0)
        self.outputNN = Output(3, 8, 5, dropout_rate=0.0)
        self.hidden = None
        return

    def call(self, x):
        output = self.outputNN(x, self.hidden)
        self.hidden = self.hiddenNN(x, self.hidden)
        return output

